# Gensim Topic Modeling, Take 2

When we started with topic modeling, we used the combined (merged) dataset as Jorge and Ilona had recommended. However, when we met with Sayan Mukherjee (6/28) (https://stat.duke.edu/people/sayan-mukherjee), he advised us to be wary of batch effects and to run LDA on the transcripts from the DVD and VA Datasets separately. Once we had our topics, we should look at the estimated term frequency within each selected topic vs overall term frequency. Should the estimated term frequency closely mirror the overall term frequency, that's not a good topic (as it's just telling us what's in the documents). For those topics that show more variation, look at the words that make up the topics and see how close these topics are.

Let's get started.

## Set Up

### Import Statements and Load Datasets

In [1]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
import numpy as np
import pandas as pd
from pprint import pprint

#gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.models import TfidfModel
from gensim.models import NormModel

#spacy for lemmatization
import spacy

#plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from collections import Counter

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/samanthagarland/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
#read in our transcripts
transcript_df = pd.read_csv("/Users/samanthagarland/Downloads/shared_data_merged.csv")

dvd = transcript_df[transcript_df["Study"] == "DVD"]
va = transcript_df[transcript_df["Study"] == "VA"]

### Methods

In [3]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))

def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stopWords] for doc in texts]

def make_bigrams_dvd(texts):
    return [bigram_mod_dvd[doc] for doc in texts]

def make_trigrams_dvd(texts):
    return [trigram_mod_dvd[bigram_mod_dvd[doc]] for doc in texts]

def make_bigrams_va(texts):
    return [bigram_mod_va[doc] for doc in texts]

def make_trigrams_dvd(texts):
    return [trigram_mod_va[bigram_mod_va[doc]] for doc in texts]

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out        

def compute_coherence_values(dictionary, corpus, texts, limit, start, step):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

### Stop Words

In [17]:
stopWords = stopwords.words('english')

stopWords = set([word.replace("'", "") for word in stopWords])

stopWords = stopWords.union(set(["mmmhmm", "mmmhmmm", "ohio", "whatev", "alrighty", "alright", "sure", "'", "'cause", "'em", 'a', 'aaah', 'aah', 'about', 'above', 'african', 'after', 'again', 'against', 'ah', 'ahh', 'ahhh', 'ahhm', 'ain', 'aint', 'alabama', 'alaska', 'all', 'am', 'an', 'anand', 'and', 'andand', 'any', 'are', 'aren', 'arent', 'as', 'at', 'ay', 'b', 'be', 'because', 'been', 'before', 'being', 'below', 'between', 'both', 'but', 'by', 'bye', 'c', 'california', 'came', 'can', 'cant', 'clean', 'costa_rica', 'could', 'couldn', 'couldnt', 'cuz', 'd', 'did', 'didn', 'didnt', 'do', 'does', 'doesn', 'doesnt', 'doin', 'doing', 'dokey', 'don', 'dont', 'down', 'during', 'e', 'each', 'eek', 'eh', 'em', 'er', 'europe', 'f', 'few', 'for', 'from', 'further', 'g', 'gal', 'gee', 'geez', 'germany', 'go', 'gonna', 'gosh', 'greek', 'h', 'ha', 'had', 'hadn', 'hadnt', 'has', 'hasn', 'hasnt', 'have', 'haven', 'havent', 'having', 'he', 'heh', 'hello', 'henry', 'her', 'here', 'hers', 'herself', 'hes', 'hey', 'hi', 'him', 'himself', 'his', 'hm', 'hmm', 'hmmm', 'hodgkins', 'how', 'huh', 'hum', 'i', 'id', 'if', 'ifif', 'ii', 'iii', 'ill', 'im', 'in', 'inaudible', 'indecipherable', 'indianapolis', 'into', 'is', 'isis', 'isn', 'isnt', 'it', 'itd', 'itit', 'itll', 'its', 'itself', 'ive', 'j', 'just', 'k', 'kay', 'kinda', 'l', 'laughs', 'le', 'legend', 'lets', 'like', 'll', 'm', 'ma', 'md', 'mdmd', 'me', 'mhm', 'mhmmm', 'michigan', 'mightn', 'mightnt', 'mightve', 'mm', 'mmhm', 'mmhmm', 'mmm', 'more', 'most', 'mustn', 'mustnt', 'my', 'myself', 'n', 'nah', 'nahuh', 'needn', 'neednt', 'no', 'nooh', 'nope', 'nor', 'not', 'now', 'o', 'of', 'off', 'oh', 'ohh', 'ohhh', 'ok', 'okay', 'okey', 'on', 'once', 'only', 'oooh', 'or', 'oth', 'other', 'othumhmm', 'oughta', 'our', 'ours', 'ourselves', 'out', 'over', 'own', 'p', 'patient', 'phi', 'physician', 'potter', 'pt', 'pt/so', 'q', 'r', 're', 'right', 's', 'same', 'shan', 'shant', 'she', 'shes', 'should', 'shouldn', 'shouldnt', 'shouldve', 'so', 'some', 'sounds', 'such', 'swedish', 't', 'th', 'than', 'that', 'thatll', 'thats', 'thatsthat', 'the', 'their', 'theirs', 'them', 'themselves', 'then', 'there', 'thered', 'theres', 'these', 'thethe', 'thew', 'they', 'theyll', 'theyre', 'this', 'those', 'through', 'to', 'too', 'u', 'uh', 'uhh', 'uhhhhh', 'uhhm', 'uhhmm', 'uhhuh', 'uhm', 'uhmhmmm', 'uhmmm', 'uhoh', 'um', 'umhmm', 'umm', 'under', 'unhunh', 'until', 'up', 'us', 'uuh', 'v', 've', 'very', 'vietnam', 'virginia', 'w', 'walsh', 'wanna', 'was', 'washington', 'wasn', 'wasnt', 'we', 'wed', 'well', 'were', 'weren', 'werent', 'weve', 'wewe', 'what', 'whatd', 'whatev', 'whatnot', 'whats', 'when', 'where', 'whew', 'which', 'while', 'who', 'whoa', 'whom', 'whos', 'why', 'will', 'with', 'won', 'wont', 'would', 'wouldn', 'wouldnt', 'x', 'y', 'ya', 'yada', 'yah', 'yea', 'yeah', 'yep', 'yeyeah', 'you', 'youd', 'youl', 'youll', 'your', 'youre', 'yours', 'yourself', 'yourselves', 'youve', 'yup', 'z']))

print("We have", len(stopWords), "stop words.")

We have 359 stop words.


### Pre Processing

In [18]:
t1 = dvd["Convo_1"]
t2 = va["Convo_1"]

data_dvd = t1.dropna()
data_va = t2.dropna()

print("We have", len(data_dvd), "conversations from DVD.")
print("We have", len(data_va), "conversations from VA.")

We have 157 conversations from DVD.
We have 257 conversations from VA.


In [19]:
data_words_dvd = list(sent_to_words(data_dvd))
data_words_va = list(sent_to_words(data_va))

##### Bigrams and Trigrams

In [20]:
# Build the bigram and trigram models
bigram_dvd = gensim.models.Phrases(data_words_dvd, min_count=2, threshold=100) 
trigram_dvd = gensim.models.Phrases(bigram_dvd[data_words_dvd], threshold=100)  

bigram_va = gensim.models.Phrases(data_words_va, min_count=2, threshold=100) 
trigram_va = gensim.models.Phrases(bigram_va[data_words_va], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod_dvd = gensim.models.phrases.Phraser(bigram_dvd)
trigram_mod_dvd = gensim.models.phrases.Phraser(trigram_dvd)

bigram_mod_va = gensim.models.phrases.Phraser(bigram_va)
trigram_mod_va = gensim.models.phrases.Phraser(trigram_va)

/Users/samanthagarland/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


##### Remove Stop Words and Lemmatize

In [21]:
#remove stop words
data_words_nostops_dvd = remove_stopwords(data_words_dvd)
data_words_nostops_va = remove_stopwords(data_words_va)

#form bigrams
data_words_bigrams_dvd = make_bigrams_dvd(data_words_nostops_dvd)
data_words_bigrams_va = make_bigrams_va(data_words_nostops_va)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized_dvd = lemmatization(data_words_bigrams_dvd, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
data_lemmatized_va = lemmatization(data_words_bigrams_va, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

# Only keep conversations with at least 10 words
data_lemmatized_dvd = [convo for convo in data_lemmatized_dvd if len(convo) > 10]
data_lemmatized_va = [convo for convo in data_lemmatized_va if len(convo) > 10]

data_lemmatized_dvd = [[word for word in convo if word not in stopWords] for convo in data_lemmatized_dvd]
data_lemmatized_va = [[word for word in convo if word not in stopWords] for convo in data_lemmatized_va]

### Create Corpus

In [22]:
id2word_dvd = corpora.Dictionary(data_lemmatized_dvd)
id2word_va = corpora.Dictionary(data_lemmatized_va)

#Filter dictionary
#keep this in, leave it out?
id2word_dvd.filter_extremes(no_below = 0.30, no_above = 0.60, keep_n = 5000, keep_tokens = None)
id2word_va.filter_extremes(no_below = 0.30, no_above = 0.60, keep_n = 5000, keep_tokens = None)


#creates corpus
texts_dvd = data_lemmatized_dvd
texts_va = data_lemmatized_va

#tdf
corpus_dvd = [id2word_dvd.doc2bow(text) for text in texts_dvd]
corpus_va = [id2word_va.doc2bow(text) for text in texts_va]

tfidf_dvd = TfidfModel(corpus_dvd)
tfidf_va = TfidfModel(corpus_va)

corpus_tfidf_dvd = tfidf_dvd[corpus_dvd]
corpus_tfidf_va = tfidf_va[corpus_va]

In [23]:
corp_dvd = corpus_dvd
corp_va = corpus_va

## General LDA Model

In [24]:
#dvd
lda_model_dvd = gensim.models.ldamodel.LdaModel(corpus=corp_dvd,
                                           id2word=id2word_dvd,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=3,
                                           chunksize=20,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
#va
lda_model_va = gensim.models.ldamodel.LdaModel(corpus=corp_va,
                                           id2word=id2word_va,
                                           num_topics=15, 
                                           random_state=100,
                                           update_every=3,
                                           chunksize=20,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

### Topic Words and Weights

In [25]:
#view these topics
print("DVD Topics:")
pprint(lda_model_dvd.print_topics())
doc_lda_dvd = lda_model_dvd[corp_dvd]
print()
print("VA Topics:")
pprint(lda_model_va.print_topics())
doc_lda_va = lda_model_va[corp_va]

DVD Topics:
[(0,
  '0.005*"multiplication" + 0.000*"seed" + 0.000*"sexual_function" + '
  '0.000*"study" + 0.000*"active_surveillance" + 0.000*"cell" + '
  '0.000*"intermediate" + 0.000*"exam" + 0.000*"surveillance" + '
  '0.000*"external_beam"'),
 (1,
  '0.008*"testosterone" + 0.006*"difference" + 0.005*"fill" + 0.005*"half" + '
  '0.005*"intermediate" + 0.005*"test" + 0.005*"study" + 0.005*"bowel" + '
  '0.004*"datum" + 0.004*"proton"'),
 (2,
  '0.000*"sample" + 0.000*"active_surveillance" + 0.000*"surveillance" + '
  '0.000*"cell" + 0.000*"likelihood" + 0.000*"class" + 0.000*"friend" + '
  '0.000*"gland" + 0.000*"recovery" + 0.000*"seed"'),
 (3,
  '0.000*"seed" + 0.000*"active_surveillance" + 0.000*"gland" + 0.000*"rectum" '
  '+ 0.000*"sample" + 0.000*"cell" + 0.000*"intermediate" + 0.000*"study" + '
  '0.000*"bowel" + 0.000*"half"'),
 (4,
  '0.000*"active_surveillance" + 0.000*"difference" + 0.000*"seed" + '
  '0.000*"sample" + 0.000*"rectum" + 0.000*"correct" + 0.000*"sexual_func

### Perplexity and Coherence

In [26]:
print("DVD:")
print('\nPerplexity: ', lda_model_dvd.log_perplexity(corp_dvd)) #a measure of how good the model is

coherence_model_lda_dvd = CoherenceModel(model=lda_model_dvd, texts=data_lemmatized_dvd, dictionary=id2word_dvd, coherence='c_v')
coherence_lda_dvd = coherence_model_lda_dvd.get_coherence()
print('\nCoherence Score: ', coherence_lda_dvd)

print()
print("VA:")
print('\nPerplexity: ', lda_model_va.log_perplexity(corp_va)) #a measure of how good the model is

coherence_model_lda_va = CoherenceModel(model=lda_model_va, texts=data_lemmatized_va, dictionary=id2word_va, coherence='c_v')
coherence_lda_va = coherence_model_lda_va.get_coherence()
print('\nCoherence Score: ', coherence_lda_va)

DVD:

Perplexity:  -7.777219173673101

Coherence Score:  0.30157640249125117

VA:

Perplexity:  -7.554736507826343

Coherence Score:  0.33185977851648224


### Visualizations

In [27]:
print("DVD:")
pyLDAvis.enable_notebook()
vis_dvd = pyLDAvis.gensim.prepare(lda_model_dvd, corp_dvd, id2word_dvd)
vis_dvd

DVD:


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
7      31.571589        1       1  0.100518  0.046412
8      15.750015        1       2  0.197417 -0.078119
10     15.385337        1       3  0.092669  0.142779
13     13.914958        1       4  0.122093  0.109935
9      10.918649        1       5  0.139585  0.135732
1       9.334164        1       6  0.135196 -0.031076
6       3.095848        1       7  0.192050 -0.224277
11      0.018586        1       8 -0.128840 -0.014969
0       0.003032        1       9 -0.121855 -0.012540
14      0.002892        1      10 -0.122081 -0.012572
4       0.000998        1      11 -0.121336 -0.012276
3       0.000993        1      12 -0.121291 -0.012294
2       0.000982        1      13 -0.121419 -0.012243
12      0.000982        1      14 -0.121327 -0.012230
5       0.000979        1      15 -0.121379 -0.012261, topic_info=     Category        Freq                 Term       Total  loglift  logprob
term                                                                        
768   Default  416.000000                study  416.000000  30.0000  30.0000
352   Default  292.000000         surveillance  292.000000  29.0000  29.0000
313   Default  424.000000                 seed  424.000000  28.0000  28.0000
174   Default  225.000000              implant  225.000000  27.0000  27.0000
147   Default  469.000000                gland  469.000000  26.0000  26.0000
499   Default  329.000000                 dose  329.000000  25.0000  25.0000
17    Default  217.000000             approach  217.000000  24.0000  24.0000
155   Default  229.000000                group  229.000000  23.0000  23.0000
581   Default  347.000000         intermediate  347.000000  22.0000  22.0000
317   Default  337.000000      sexual_function  337.000000  21.0000  21.0000
454   Default  401.000000                 cell  401.000000  20.0000  20.0000
979   Default  312.000000             recovery  312.000000  19.0000  19.0000
724   Default  328.000000               sample  328.000000  18.0000  18.0000
338   Default  314.000000               spread  314.000000  17.0000  17.0000
712   Default  378.000000               rectum  378.000000  16.0000  16.0000
988   Default  299.000000              robotic  299.000000  15.0000  15.0000
2141  Default  138.000000        brachytherapy  138.000000  14.0000  14.0000
1924  Default  139.000000             external  139.000000  13.0000  13.0000
658   Default  193.000000              outcome  193.000000  12.0000  12.0000
600   Default  223.000000           likelihood  223.000000  11.0000  11.0000
162   Default  202.000000                 heal  202.000000  10.0000  10.0000
8     Default  253.000000               affect  253.000000   9.0000   9.0000
437   Default  233.000000                bowel  233.000000   8.0000   8.0000
520   Default  113.000000            extensive  113.000000   7.0000   7.0000
1244  Default  127.000000              deliver  127.000000   6.0000   6.0000
510   Default  219.000000             erectile  219.000000   5.0000   5.0000
580   Default  135.000000           interested  135.000000   4.0000   4.0000
300   Default  149.000000                robot  149.000000   3.0000   3.0000
64    Default  196.000000               common  196.000000   2.0000   2.0000
4     Default  451.000000  active_surveillance  451.000000   1.0000   1.0000
...       ...         ...                  ...         ...      ...      ...
4     Topic15    0.000216  active_surveillance  451.561188  -3.0211  -8.5000
724   Topic15    0.000215               sample  328.558685  -2.7067  -8.5036
712   Topic15    0.000215               rectum  378.792725  -2.8495  -8.5041
153   Topic15    0.000215                grade  250.038818  -2.4346  -8.5045
41    Topic15    0.000214                 body  272.508972  -2.5211  -8.5050
581   Topic15    0.000214         intermediate  347.665100  -2.7653  -8.5056
941   Topic15    0.000214           

##### Fifteen Topics, Filters at 0.20 to 0.70

Tfidf:
- A cursory examination of these topics does not show any topic with variation in estimated term frequency vs overall term frequency when using the tfidf corpus.

Corpus:
- Topic 1
    - seed, study, rectum, basically, active_surveillance, gland, therapy, dose, exactly, intermediate, cell, external_beam, implant, difference, die, sexual_function, group, tend, half, likely, correct, body, read, move, often, later, bowel, better, second, urologist
    - ==> RADIATION
- Topic 2
    - surveillance, nerve, recovery, robotic, approach, sample, leakage, erectile, gland, tend, cure, grade, likelihood, operation, certainly, ahead, cell, recover, therapy, hard, quite, tissue, someone, spread, obviously, hell, sexual_function, deal, important, extensive
    - ==> side effects? surgery?
- Topic 3
    - heart, lung, listen, lump, medicine, bump, folk, belly, righty, pull, dad, seat, table, groin_area, leg, open, absolutely, curtain, card, bring, enough, lie, smoke, alright, heart_attack, ahead, exam, correct, anyway, perfect
    - ==> life stuff?
- Topic 4
    - book, schedule, erectile_dysfunction, care, walk, read, pay, write, family, set, close, mkay, bowel, one, answer, bathroom, wear, better, everybody, medication, hold, pee, enema, hopefully, run, nerve, drive, deal, person, kid
    - ==> life stuff?
- Topic 5
    - tissue, heal, small_incision, intermediate, result, list, anyway, dry, robot, appointment, card, pill, sew, heart, erectile_dysfunction, spare, advantage, certainly, belly, regain, schedule, obviously, hernia, almost_dry, listen, form, common, affect, radiate, someone
    - ==> SURGERY

##### Six Topics, Filters at 0.20-0.70

Tfidf:
- Topics don't show variation in estimated term frequency (and some overall term frequencies are extremely low).

Corpus:
- Topic 1:
    - nerve, surveillance, recovery, robotic, therapy, cell, leakage, approach, sample, tend, gland, erectile, active_surveillance, basically, tissue, certainly, cure, recover grade, hard, ahead, spread, important, sexual_function, likelihood, operation, deal, hell, obviously, quite
    - ==> SURGERY
- Topic 2:
    - study, active_surveillance, rectum, exactly, difference, dose, therapy, basically, implant, die, group, likely, sexual_function, gland, seed, read, intermediate, better, half, bowel, body, urologist, external_beam, tend, test, later, correct, often, someone, cell
    - ==> RADIATION
- Topic 3:
    - heart, listen, lung, tissue, heal, medicine, belly, card, anyway, pill, appointment, lump, small_incision, read, folk, list, result, erectile_dysfunction, certainly, alright, bump, dry, intermediate, pull, enough, wife, spare, form, hernia, feel_free
    - ==> surgery? body parts?
- Topic 4: 
    - seed, gland, dose, rectume, intermediate, move, external_beam, table, external, choose, half, cell, implant, ....
    - ==> radiation?
    
##### Fifteen Topics, Filters at 0.30-0.60

Tfidf:
- The term frequencies don't show much change (either estimated > overall, or they're stable)

Corpus:
- Topic 1:
    - correct, sample, active_surveillance, effect, robotic, cell, spread, rectum, answer, body, erectile_dysfunction, erectile, schedule, medication, exam, important, matter, book, grade, test, affect, volume, real, tumor, intermediate, hell, generally, percent, reasonable, gland
    - ==> treatment?
- Topic 2:
    - seed, study, dose, rectum, gland, implant, intermediate, group, external_beam, half, sexual_function, difference, active_surveillance, urologist, bowel, brachytherapy, external, deliver, cell, testosterone, beam, minute, needle, body, outcome, hormone, test, irritation, mmmhmmm, common
    - ==> radiation and hormone therapy
- Topic 3:
    - surveillance, approach, recovery, gland, sample, likelihodd, cure, extensive, sexual_function, grade, recover, period, choose, active_surveillance, operation, robotic, important, consider, erectile, potential, answer, spread, dad, potentially, healthy, diagnose, exam, avoid, followup, prostatectomy
    - ==> surgery?
- Topic 4:
    - mmmhmmm, cell, active_surveillance, sexual_function, robotic, affect, close, somebody, general, datum, lymph_node, become, recover, develop, hell, difference, real, external_beam, robot, local, test, recovery, longterm, incision, penis, recurrence, potentially, have, expect, lose
    - ==> treatment?
- Topic 5:
    - medicine, lump, belly, heal, small_incision, bump, pill, feel_free, righty, folk, dry, alright, pull, table, almost_dry, hernia, groin_area, smoke, list, sew, advantage, press, seat, regain, dad, wife, bother, completely_dry, leg, mine
    - ==> body parts

##### Six Topics, Filters at 0.30-0.60

Tfidf:
- No clear topics.

Corpus:
- Topic 1:
    - seed, study, rectume, dose, gland, intermediate, implant, active_surveillance, group, half, external_beam, difference, sexual_function, bowel, body, cell, urologist, correct, minute, test, external, spread, effect, plan, horomone, brachytherapy, large, deliver, common, beam
    - ==> RADIATION
- Topic 2:
    - cell, robotic, active_surveillance, correct, affect, hell, mmmhmmm, recovery, erectile, somebody, close, erectile_dysfunction, spread, general, real, lymph_node, robot, book, test, sexual_function, recover, medication, important, become, difference, operation, family, body, recurrence, everybody
    - ==> surgery
- Topic 3:
    - surveillance, sample, approach, gland, recovery, likelihood, cure, extensive, grade, answer, choose, sexual_function, important, period, operation, robotic, recover, potential, erectile, spread, consider, exam, diagnose, volume, followup, active_surveillance, medication, effect, potentially, test
    - ==> active surveillance?
- Topic 4:
    - heal, medicine, small_incision, lump, belly, pill, list, feel_free, folk, righty, bump, dry, alright, erectile_dysfunction, spare, sew, regain, pull, almost_dry, groin_area, completely_dry, table, advantage, seat, press, mine, allow, bring, smoke, notebook
    - ==> ???

In [28]:
print("VA:")
pyLDAvis.enable_notebook()
vis_va = pyLDAvis.gensim.prepare(lda_model_va, corp_va, id2word_va)
vis_va

VA:


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
8      36.522514        1       1  0.165550  0.030425
1      17.896955        1       2  0.187584  0.040400
13     16.100676        1       3  0.198007  0.001961
14     11.129534        1       4  0.189094  0.087044
4       7.955991        1       5  0.145654 -0.084451
0       3.397213        1       6  0.084513 -0.124342
6       3.249901        1       7  0.050205  0.056483
5       2.165337        1       8  0.008001  0.018451
11      1.430272        1       9 -0.093635 -0.216827
9       0.063323        1      10 -0.168072  0.036384
7       0.060907        1      11 -0.167825  0.037930
3       0.013500        1      12 -0.151711  0.029824
2       0.011131        1      13 -0.151150  0.029924
10      0.001385        1      14 -0.148107  0.028396
12      0.001374        1      15 -0.148109  0.028397, topic_info=     Category        Freq                  Term       Total  loglift  logprob
term                                                                         
17    Default  223.000000             certainly  223.000000  30.0000  30.0000
774   Default  400.000000               robotic  400.000000  29.0000  29.0000
110   Default  407.000000             typically  407.000000  28.0000  28.0000
414   Default  297.000000               benefit  297.000000  27.0000  27.0000
27    Default  395.000000                   die  395.000000  26.0000  26.0000
48    Default  300.000000                 heart  300.000000  25.0000  25.0000
43    Default  610.000000                 grade  610.000000  24.0000  24.0000
220   Default  391.000000          incontinence  391.000000  23.0000  23.0000
10    Default  541.000000             basically  541.000000  22.0000  22.0000
559   Default  288.000000          surveillance  288.000000  21.0000  21.0000
1118  Default  158.000000                  lung  158.000000  20.0000  20.0000
301   Default  414.000000             procedure  414.000000  19.0000  19.0000
106   Default  461.000000                  term  461.000000  18.0000  18.0000
363   Default  330.000000               symptom  330.000000  17.0000  17.0000
4386  Default  212.000000           doctor_name  212.000000  16.0000  16.0000
498   Default  383.000000                  open  383.000000  15.0000  15.0000
119   Default  390.000000                affect  390.000000  14.0000  14.0000
366   Default  395.000000                  test  395.000000  13.0000  13.0000
939   Default  341.000000                  read  341.000000  12.0000  12.0000
242   Default  140.000000            likelihood  140.000000  11.0000  11.0000
12    Default  137.000000                 bring  137.000000  10.0000  10.0000
841   Default  352.000000                volume  352.000000   9.0000   9.0000
125   Default  289.000000               already  289.000000   8.0000   8.0000
368   Default  312.000000                tissue  312.000000   7.0000   7.0000
1044  Default  348.000000            understand  348.000000   6.0000   6.0000
275   Default  287.000000            oncologist  287.000000   5.0000   5.0000
419   Default  147.000000                 bowel  147.000000   4.0000   4.0000
222   Default  328.000000           information  328.000000   3.0000   3.0000
1353  Default  339.000000                 tumor  339.000000   2.0000   2.0000
218   Default  253.000000                  idea  253.000000   1.0000   1.0000
...       ...         ...                   ...         ...      ...      ...
10    Topic15    0.000313             basically  541.401245  -3.1695  -8.5166
110   Topic15    0.000313             typically  407.974487  -2.8868  -8.5168
220   Topic15    0.000312          incontinence  391.629028  -2.8460  -8.5169
106   Topic15    0.000312                  term  461.709106  -3.0106  -8.5169
119   Topic15    0.000312                affect  390.604248  -2.8434  -8.5169
363   Topic15    0.000312               symptom  330.434296  -2.6761  -8.51

##### Fifteen Topics, FIlters at 0.20-0.70
Tfidf:
- Topic 3:
    - associate, weight, record, sir, belly, frequently, cardiologist, gland, increase, penis, category, specific, measure, totally, curably 
    - blood_transfusion, scare, seed_implant, colon, center, interferon, phone, hepatitis, testicle, regular, whenever, carry, actively, liver, life_span
    - ==> treatment
- Topic 4:
    - guess, shrink, plavix, clinic, lose, needle, news, visit, tube, antibiotic, group, rectal, necessary, opinion
    - cause, third, rise, sometime, swell, primary_care, minimally_invasive, region, evidence, rapidly, overall_health, instrument, late, jump, machine, reading
    - ???

Corpus:
- Topic 1:
    - sort, leakage, urinary, core, because, medication, surveillance, spread, grade, tumor, issue, base, keep, likely, pee, typically, symptom, doctor_name, area, worry, heart, erectile_function, consider, half, hospital, afterwards, appointment, incision, concern, schedule
    - ==> side effects?
- Topic 2:
    - nerve, seed, robotic, because, procedure, basically, case, grade, open, robot, catheter, control, rectum, hear, cell, young, affect, urethra, second, therapy, volume, cut, follow, least, issue, sir, incision, leak, effect, hard
    - ==> SURGERY
- Topic 3: 
    - test, grade, affect, exactly, keep, die, read, term, guy, patient, volume, benefit, life, understand, information, meet, repeat, sort, thank, because, core, amount, last, study, grow, age, positive, certainly, nice, mind
    - ==> ???
- Topic 4:
    - intermediate, patient, far, oncologist, therapy, catheter, erectile_dysfunction, hour, incontinence, young, benefit, someone, oncology, area, prostatectomy, leak, hospital, nerve, home, wait, else, open, later, away, positive, ask, hormone, surgeon, die, procedure
    - ==> SIDE EFFECTS
- Topic 5:
    - far, stuff, last, ago, basically, whatev, ever, follow, exactly, pain, depend, operation, cut, grow, pill, guess, tissue, seem, core, drink, eat, already, rather, area, least, die, lung, miss, part, psoriasis
    - ==> surgery?
- Topic 6:
    - guy, typically, great, term, tissue, tend, set, far, home, stuff, dry, job, surgeon, everybody, folk, somewhere, meet, exactly, else, middle, figure, incision, enough, available, later, leak, have, healthy, appointment, recover
    - ==> surgery?

##### Six Topics, Filters at 0.20-0.70

Tfidf: 
- Estimated term frequencies are either through the roof or near zero.

Corpus:
- Topic 1: 
    - because, nerve, grade, basically, robotic, case, procedure, guy, leakage, sort, open, spread, leak, appointment, ever, follow, least, area, keep, test, home, cure, catheter, incision, urinary, hear, part, stuff, symptom, many
    - ==> side effects? surgery?
- Topic 2:
    - intermediate, far, patient, guy, therapy, benefit, term, oncologist, die, incontinence, catheter, erectile_dysfunction, home, hour, young, meet, area, wait, else, positive, nerve, set, core, sort, hospital, affect, oncology, last, ask, meaning
    - ==> side effects?
- Topic 3:
    - sort, surveillance, core, urinary, tumor, heart, issue, keep, leakage, amount, medication, grade, base, worry, spread, concern, test, consider, thank, likely, wait, involve, term, repeat, diagnose, order, last, therapy, sit, doctor_name
    - ==> surveillance?
- Topic 4:
    - seed, volume, issue, exactly, brachytherapy, grade, plavix, situation, operation, affect, base, oncologist, ...
    - ==> radiation
    
##### Fifteen Topics, Filters at 0.30-0.60

Tfidf:
- Topic 4:
    - seed, medicine, impact, effective, shrink, general, total, cardiologist, include, advanced, seat
    - ==> ???
- Topic 5:
    - pee, slightly, improve, stable, repeat, blood_loss, valve, pad, computer, safe, approach
    - ==> ???

Corpus:
- Topic 1:
    - procedure, robotic, grade, open, patient, cure, incontinence, meet, erectile_dysfunction, base, erectile_function, rectum, cell, road, oncologist, afterwards, someone, definitely, urethra, information, medication, testerone, correct, affect, control, sit, positive, normal, schedule, volume
    - ==> SURGERY
- Topic 2:
    - tumor, surveillance, heart, basically, pee, test, seed, amount, base, pain, half, worry, likely, guess, grade, medication, medicine, many, symptom, seem, bleed, volume, cut, often, concern, schedule, ago, depend, night, order
    - ==> ???
- Topic 3:
    - basically, grade, die, test, understand, grow, read, term, repeat, life, information, situation, depend, affect, robot, trouble, thank, seed, whatev, stage, news, kill, potentially, open, effect, book, enough, slow_grow, figure, perfect
    - ==> diagnostics
- Topic 4:
    - term, typically, tend, tissue, folk, certainly, oncologist, somebody, lymph_node, somewhere, middle, later, seed, erectile_function, dry, hard, likelihood, base, everybody, life, stop, positive, happy, together, whatev, initially, approve, job, hour, consider
    - ==> ???
- Topic 5:
    - patient, benefit, die, incontinence, affect, erectile_dysfunction, brain, meet, term, mind, meaning, slow_grow, understand, choose, survival, correct, lifespan, answer, compare, negative, study, hormone, immune_system, oncologist, example, alive, quality, outcome, friday, pancreatic
    - ==> side effects/outcomes
    
##### Six Topics, Filters at 0.30-0.60

Tfidf:
- No clear topics.

Corpus:
- Topic 1:
    - basically, robotic, open, term, cut, grade, control, depend, tissue, robot, die, already, understand, procedure, seed, cure, grow, whatev, ago, base, information, kill, cell, typically, affect, life, sit, situation, rectum, enough
    - ==> treatment
- Topic 2:
    - grade, tumor, surveillance, test, doctor_name, amount, incontinence, likely, volume, erectile_function, patient, base, seed, symptom, medication, schedule, many, pee, procedure, heart, worry, concern, guess, half, read, afterwards, normal, affect, consider, testosterone
    - ==> diagnostics
- Topic 3:
    - oncologist, hour, require, oncology, medication, positive, gland, heart, consider, erectile_dysfunction, relate, bowel, base, essentailly, ...
    - ==> ???
- Topic 4:
    - patient, benefit, die, incontinence, affect, erectile_dysfunction, hormone, term, meet, ....
    - ==> ???

## LDA Mallet

In [29]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = "/Users/samanthagarland/Downloads/mallet-2.0.8/bin/mallet" # update this path
ldamallet_dvd = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corp_dvd, num_topics=6, id2word=id2word_dvd)
ldamallet_va = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corp_va, num_topics=6, id2word=id2word_va)


### Topic Words and Weights

In [30]:
print("DVD:")
pprint(ldamallet_dvd.show_topics(formatted=True))
print()
print("VA:")
pprint(ldamallet_va.show_topics(formatted=True))

DVD:
[(0,
  '0.016*"heal" + 0.011*"medicine" + 0.011*"belly" + 0.009*"list" + '
  '0.009*"pill" + 0.009*"folk" + 0.009*"spare" + 0.008*"dry" + '
  '0.008*"small_incision" + 0.007*"hernia"'),
 (1,
  '0.025*"active_surveillance" + 0.011*"difference" + 0.009*"sexual_function" '
  '+ 0.007*"reasonable" + 0.007*"sample" + 0.007*"average" + 0.007*"study" + '
  '0.007*"outcome" + 0.007*"benefit" + 0.007*"urologist"'),
 (2,
  '0.013*"robotic" + 0.010*"hell" + 0.010*"correct" + 0.009*"cell" + '
  '0.008*"real" + 0.008*"recover" + 0.008*"close" + 0.008*"book" + 0.008*"dad" '
  '+ 0.008*"erectile_dysfunction"'),
 (3,
  '0.011*"medication" + 0.010*"exam" + 0.009*"urinate" + 0.008*"morning" + '
  '0.008*"blood_pressure" + 0.008*"stop" + 0.007*"bathroom" + 0.007*"turn" + '
  '0.006*"sleep" + 0.006*"urination"'),
 (4,
  '0.022*"recovery" + 0.019*"surveillance" + 0.015*"approach" + 0.014*"sample" '
  '+ 0.014*"gland" + 0.013*"grade" + 0.013*"cure" + 0.012*"likelihood" + '
  '0.009*"operation" + 0.008*

### Perplexity and Coherence

In [31]:
# Compute Coherence Score
print("DVD:")
coherence_model_ldamallet_dvd = CoherenceModel(model=ldamallet_dvd, texts=data_lemmatized_dvd, dictionary=id2word_dvd, coherence='c_v')
coherence_ldamallet_dvd = coherence_model_ldamallet_dvd.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet_dvd)
print()
print("VA:")
coherence_model_ldamallet_va = CoherenceModel(model=ldamallet_va, texts=data_lemmatized_va, dictionary=id2word_va, coherence='c_v')
coherence_ldamallet_va = coherence_model_ldamallet_va.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet_va)

DVD:

Coherence Score:  0.37141713573729546

VA:

Coherence Score:  0.3297085264877492


### Visualizations

In [32]:
print("DVD")
model_dvd = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet_dvd)
pyLDAvis.enable_notebook()
viz_dvd = pyLDAvis.gensim.prepare(model_dvd, corp_dvd, id2word_dvd)
viz_dvd

DVD


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
5      21.020218        1       1 -0.001017  0.000596
1      19.947314        1       2  0.000638  0.000200
3      17.419466        1       3  0.000264 -0.000340
2      16.551592        1       4 -0.000210 -0.000819
4      13.474378        1       5  0.000509  0.000668
0      11.587032        1       6 -0.000184 -0.000306, topic_info=     Category       Freq                 Term      Total  loglift  logprob
term                                                                      
914   Default  20.000000             identify  20.000000  30.0000  30.0000
360   Default  21.000000               timely  21.000000  29.0000  29.0000
1198  Default  21.000000            tolerable  21.000000  28.0000  28.0000
580   Default  22.000000           interested  22.000000  27.0000  27.0000
3975  Default  22.000000             modality  22.000000  26.0000  26.0000
309   Default  22.000000                scale  22.000000  25.0000  25.0000
4158  Default  21.000000             perceive  21.000000  24.0000  24.0000
4951  Default  20.000000     specific_antigen  20.000000  23.0000  23.0000
1666  Default  20.000000                valid  20.000000  22.0000  22.0000
2216  Default  21.000000             bike_rid  21.000000  21.0000  21.0000
3821  Default  21.000000      highly_unlikely  21.000000  20.0000  20.0000
3782  Default  20.000000              fixable  20.000000  19.0000  19.0000
4490  Default  24.000000                 dual  24.000000  18.0000  18.0000
2010  Default  23.000000                 vary  23.000000  17.0000  17.0000
3660  Default  20.000000         interruption  20.000000  16.0000  16.0000
3897  Default  23.000000                 lawn  23.000000  15.0000  15.0000
1048  Default  21.000000                 baby  21.000000  14.0000  14.0000
4872  Default  20.000000                alarm  20.000000  13.0000  13.0000
1344  Default  21.000000                  tap  21.000000  12.0000  12.0000
4376  Default  21.000000                bride  21.000000  11.0000  11.0000
1734  Default  22.000000                track  22.000000  10.0000  10.0000
2600  Default  22.000000        effectiveness  22.000000   9.0000   9.0000
15    Default  21.000000              anymore  21.000000   8.0000   8.0000
4419  Default  20.000000           synthesize  20.000000   7.0000   7.0000
3260  Default  21.000000                handy  21.000000   6.0000   6.0000
2602  Default  21.000000                 gain  21.000000   5.0000   5.0000
2956  Default  22.000000          controlwise  22.000000   4.0000   4.0000
4646  Default  21.000000                blade  21.000000   3.0000   3.0000
1953  Default  20.000000            literally  20.000000   2.0000   2.0000
3142  Default  20.000000           infiltrate  20.000000   1.0000   1.0000
...       ...        ...                  ...        ...      ...      ...
2956   Topic6   3.233110          controlwise  22.511097   0.2147  -8.2628
4130   Topic6   3.084682                italy  21.479713   0.2146  -8.3098
2305   Topic6   3.064530                polyp  21.376332   0.2129  -8.3164
953    Topic6   3.023719                  nno  21.103961   0.2123  -8.3298
4211   Topic6   3.013045                movie  21.031846   0.2122  -8.3333
978    Topic6   3.131014              recover  21.929018   0.2088  -8.2949
80     Topic6   3.044637                  dad  21.347275   0.2077  -8.3229
3218   Topic6   3.096085           girlfriend  21.741805   0.2062  -8.3061
877    Topic6   3.201413       effect_profile  22.569666   0.2023  -8.2727
3242   Topic6   2.917511                  air  20.577695   0.2018  -8.3655
4872   Topic6   2.852589                alarm  20.137229   0.2009  -8.3881
2751   Topic6   3.001018  excellent_prognosis  21.200497   0.2002  -8.3373
491    Topic6   3.166286           difference  22.435710   0.1972  -8.2837
3364   Topic6   3.085934            calibrate  21.890320   0.1961  -8.3094


##### Fifteen Topics, Filters at 0.20-0.70
Tfidf:
- No topics have variation in estimated term frequency and overall term frequency.
    - Visualization: estimated term frequency > overall term frequency

Corpus:
- No topics have variation in estimated term frequency and overall term frequency.
    - Visualization: estimated term frequency < overall term frequency

##### Six Topics, Filters at 0.20-0.70
Tfidf:
- Same, no topics have variation (estimated > overall).

Corpus:
- Same, no topics have variation.

In [33]:
print("VA")
model_va = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet_va)
pyLDAvis.enable_notebook()
viz_va = pyLDAvis.gensim.prepare(model_va, corp_va, id2word_va)
viz_va

VA


PreparedData(topic_coordinates=            Freq  cluster  topics         x         y
topic                                                
4      18.062982        1       1 -0.000302 -0.000665
5      17.504703        1       2 -0.000054  0.000856
1      17.240047        1       3  0.000503 -0.000218
2      17.040063        1       4  0.000497  0.000489
0      15.461306        1       5  0.000414 -0.000527
3      14.690900        1       6 -0.001059  0.000066, topic_info=     Category       Freq                    Term      Total  loglift  logprob
term                                                                         
654   Default  22.000000                 excited  22.000000  30.0000  30.0000
2768  Default  22.000000                printout  22.000000  29.0000  29.0000
4563  Default  23.000000            clear_liquid  23.000000  28.0000  28.0000
195   Default  23.000000                    fall  23.000000  27.0000  27.0000
4982  Default  23.000000                  errand  23.000000  26.0000  26.0000
3631  Default  22.000000          congratulation  22.000000  25.0000  25.0000
2999  Default  22.000000                rigorous  22.000000  24.0000  24.0000
2797  Default  23.000000                  reduce  23.000000  23.0000  23.0000
4231  Default  22.000000                  racing  22.000000  22.0000  22.0000
4330  Default  22.000000                   lasix  22.000000  21.0000  21.0000
2928  Default  24.000000               lighthead  24.000000  20.0000  20.0000
3054  Default  23.000000             equilibrium  23.000000  19.0000  19.0000
1975  Default  22.000000                   radia  22.000000  18.0000  18.0000
1082  Default  22.000000                nowadays  22.000000  17.0000  17.0000
898   Default  23.000000               guarantee  23.000000  16.0000  16.0000
436   Default  21.000000                    cork  21.000000  15.0000  15.0000
2590  Default  22.000000                forecast  22.000000  14.0000  14.0000
1866  Default  22.000000                  divide  22.000000  13.0000  13.0000
3194  Default  22.000000               statement  22.000000  12.0000  12.0000
4345  Default  22.000000             wonderfully  22.000000  11.0000  11.0000
3383  Default  21.000000               fee_based  21.000000  10.0000  10.0000
1620  Default  24.000000                     els  24.000000   9.0000   9.0000
4275  Default  22.000000                    dust  22.000000   8.0000   8.0000
4416  Default  21.000000               objective  21.000000   7.0000   7.0000
1075  Default  22.000000             manufacture  22.000000   6.0000   6.0000
1602  Default  21.000000                 combine  21.000000   5.0000   5.0000
4750  Default  23.000000                coincide  23.000000   4.0000   4.0000
3019  Default  23.000000            gall_bladder  23.000000   3.0000   3.0000
4601  Default  23.000000  robotic_prostatectomie  23.000000   2.0000   2.0000
2747  Default  24.000000          small_fraction  24.000000   1.0000   1.0000
...       ...        ...                     ...        ...      ...      ...
3383   Topic6   3.990581               fee_based  21.956865   0.2128  -8.3393
4567   Topic6   3.983177                   juice  21.916539   0.2128  -8.3412
4275   Topic6   4.033186                    dust  22.197182   0.2125  -8.3287
3302   Topic6   4.136620                 biggest  22.849549   0.2089  -8.3034
2768   Topic6   4.076960                printout  22.535483   0.2082  -8.3179
161    Topic6   4.130511               concerned  22.877455   0.2062  -8.3049
4670   Topic6   4.013257                favorite  22.230270   0.2061  -8.3337
3334   Topic6   4.034036               wonderful  22.479809   0.2001  -8.3285
2625   Topic6   4.065207              milliliter  22.661136   0.1998  -8.3208
2946   Topic6   4.320093             symptomatic  24.100401   0.1990  -8.2600
185    Topic6   4.068023                  effect  22.712895   0.1982  -8.3201
1218   Topic6   4.105030                   throw  22.923977   0.1980  -8.3110
922    Topic6  

##### Fifteen Topics, Filters at 0.20-0.70
Tfidf: 
- Again, no topic showed variation between estimated term frequency and overall term frequency.

Corpus: 
- No topic showed variation (estimated term frequency was constant and low across all words in all topics).

##### Six Topics, Filters at 0.20-0.70
Tfidf:
- No topics showed variation.

Corpus:
- No topics showed variation.

## Observations:

- Gensim general model >> Mallet model
- corpus >> tf-idf corpus
- might just be me, but the topics seem clearer in the DVD model